In [1]:
import plotly.offline as pyo
import plotly_express as pyx
import plotly.graph_objects as go
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder
import plotly.figure_factory as ff
import numpy as np
from numpy import *
import pandas as pd

data = pd.read_csv("sample_semanai.csv")
data = data.drop_duplicates().iloc[:]
top_10 = data.groupby(['Clave platillo'])['Cantidad'].agg(
    {"Cuenta": len}).sort_values(
    "Cuenta", ascending=False).head(10).reset_index()
top_10 = top_10['Clave platillo']

/home/kaetahr/.local/lib64/python3.6/site-packages/ipykernel_launcher.py:15: FutureWarning:

using a dict on a Series for aggregation
is deprecated and will be removed in a future version. Use                 named aggregation instead.

    >>> grouper.agg(name_1=func_1, name_2=func_2)




In [28]:
data_stacked = data.groupby(['Día', 'Grupo'])['Cantidad'].sum().reset_index().sort_values(['Día','Cantidad'],ascending=False)
fig = pyx.bar(data_stacked, x='Día', y='Cantidad', color='Grupo', category_orders = {
    'Día' : ['lunes', 'martes', 'miércoles', 'jueves', 'viernes', 'sábado', 'domingo' ]
})
fig.show()

In [13]:
data_stacked = data.groupby(['Clave platillo', 'Grupo'])['Cantidad'].sum().reset_index().sort_values(['Clave platillo','Cantidad'],ascending=False)
fig = pyx.bar(data_stacked, x='Grupo', y='Cantidad', color = 'Clave platillo')
fig.show()

In [4]:
data2 = data[data["Acción"] != "Cancelación"]
data2.dropna(axis=0, subset=['Clave platillo'], inplace=True)
data2 = data2[['Cantidad','Clave platillo', 'idEncuesta']]
data2 = data2.dropna()

In [5]:
data2 = data2.groupby(['idEncuesta', 'Clave platillo'])['Cantidad'].sum().unstack().reset_index().fillna(0).set_index('idEncuesta')

In [6]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

data2 = data2.applymap(encode_units)
data2 = data2.loc[:, data2.columns.intersection(top_10)]
data2 = data2[(data2.T != 0).any()]
data2

Clave platillo,BAS1,BAS50,BCA8,CLA1,HAM1,HAM12,HAM7,MBCA63,MCLÁ25,MHAM30
idEncuesta,,,,,,,,,,
115788,0,0,0,0,0,0,0,0,0,1
115789,0,0,0,0,1,0,0,0,0,0
117530,0,0,1,0,0,0,0,0,0,0
117533,0,0,0,0,0,0,0,0,0,1
117536,0,1,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...
615342,0,0,1,0,1,0,0,0,0,0
615470,0,0,0,0,1,0,0,0,0,0
615569,0,0,1,0,0,1,0,0,0,0


In [7]:
datatemp1 = data2[data2[top_10] == 1].groupby(top_10[1]).sum().reset_index().reindex(sorted(data2.columns), axis=1)
for i in range(8):
    datatemp2 = data2[data2[top_10] == 1].groupby(top_10[i+2]).sum().reset_index().reindex(sorted(data2.columns), axis=1)
    datatemp1 = datatemp1.append(datatemp2)
datatemp1

Clave platillo,BAS1,BAS50,BCA8,CLA1,HAM1,HAM12,HAM7,MBCA63,MCLÁ25,MHAM30
0,1.0,151.0,142.0,192.0,178.0,316.0,177.0,123.0,136.0,783.0
0,316.0,204.0,168.0,97.0,112.0,1.0,80.0,190.0,72.0,710.0
0,123.0,97.0,124.0,132.0,130.0,190.0,94.0,1.0,81.0,531.0
0,142.0,84.0,1.0,133.0,128.0,168.0,90.0,124.0,73.0,473.0
0,151.0,1.0,84.0,98.0,124.0,204.0,103.0,97.0,63.0,443.0
0,178.0,124.0,128.0,73.0,1.0,112.0,53.0,130.0,39.0,486.0
0,192.0,98.0,133.0,1.0,73.0,97.0,63.0,132.0,197.0,287.0
0,177.0,103.0,90.0,63.0,53.0,80.0,1.0,94.0,40.0,396.0
0,136.0,63.0,73.0,197.0,39.0,72.0,40.0,81.0,1.0,168.0


In [8]:
datatemp1 = datatemp1.reset_index()
datatemp1

Clave platillo,index,BAS1,BAS50,BCA8,CLA1,HAM1,HAM12,HAM7,MBCA63,MCLÁ25,MHAM30
0,0,1.0,151.0,142.0,192.0,178.0,316.0,177.0,123.0,136.0,783.0
1,0,316.0,204.0,168.0,97.0,112.0,1.0,80.0,190.0,72.0,710.0
2,0,123.0,97.0,124.0,132.0,130.0,190.0,94.0,1.0,81.0,531.0
3,0,142.0,84.0,1.0,133.0,128.0,168.0,90.0,124.0,73.0,473.0
4,0,151.0,1.0,84.0,98.0,124.0,204.0,103.0,97.0,63.0,443.0
5,0,178.0,124.0,128.0,73.0,1.0,112.0,53.0,130.0,39.0,486.0
6,0,192.0,98.0,133.0,1.0,73.0,97.0,63.0,132.0,197.0,287.0
7,0,177.0,103.0,90.0,63.0,53.0,80.0,1.0,94.0,40.0,396.0
8,0,136.0,63.0,73.0,197.0,39.0,72.0,40.0,81.0,1.0,168.0


In [9]:
datatemp1 = datatemp1[['BAS1', 'HAM12', 'MBCA63', 'BCA8', 'BAS50', 'HAM1','CLA1', 'HAM7', 'MCLÁ25']]
datatemp_tot = datatemp1.copy()
datatemp_tot['Total'] = datatemp_tot.sum(axis=1).values - 1
top_10 = top_10[1:]
datatemp_tot.set_index(top_10,inplace =True)
datatemp1.style.background_gradient(cmap='Blues')



Clave platillo,BAS1,HAM12,MBCA63,BCA8,BAS50,HAM1,CLA1,HAM7,MCLÁ25
0,1,316,123,142,151,178,192,177,136
1,316,1,190,168,204,112,97,80,72
2,123,190,1,124,97,130,132,94,81
3,142,168,124,1,84,128,133,90,73
4,151,204,97,84,1,124,98,103,63
5,178,112,130,128,124,1,73,53,39
6,192,97,132,133,98,73,1,63,197
7,177,80,94,90,103,53,63,1,40
8,136,72,81,73,63,39,197,40,1


In [10]:
values_list = datatemp1.values.tolist()
productos = ['BAS1', 'HAM12', 'MBCA63', 'BCA8', 'BAS50', 'HAM1','CLA1', 'HAM7', 'MCLÁ25']
fig = go.Figure(data=go.Heatmap(
                    x=productos,
                    y=productos,
                    z=values_list))
fig.show()

In [11]:
df_por_edades= data['¿Cuál es el rango de edades de tu mesa?'].str.get_dummies(" ")
df_con_quien= data['El día de hoy ¿Con quién nos visita?'].str.get_dummies(" ")
df_grupos= data['Grupo'].str.get_dummies(" ")


KeyError: '¿Cuál es el rango de edades de tu mesa?'